In [1]:
from models.constants import RANDOM_STATE
from process_and_ml.data_wrangler import CleanUp, load_azdias, load_customers, load_test_file
from process_and_ml.where_is import WhereIs

In [2]:

where_is = WhereIs(False)

In [3]:
all_paths = where_is.get_paths_list

In [4]:
#load azdias(population info)
azdias_df = load_azdias(all_paths[0], sample_ratio=0.2)


/home/jair/.cache/pypoetry/virtualenvs/arvatocapstoneproject-L5ipTb8G-py3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
customers_df = load_customers(all_paths[1], sample_ratio=0.2)

In [6]:
test_file, lnr = load_test_file(all_paths[5])

In [7]:
clean_up = CleanUp(all_paths)



In [8]:
argument_dict = {'azdias':azdias_df,'customers_df':customers_df, 'test_file': test_file}

In [9]:
# Mount dict for pipeline, you can check on pipeline documentation
dfs_dict = clean_up.pipeline_clean_up(argument_dict,unknowns_df=all_paths[4], info_df=all_paths[8],attr_df=all_paths[7])

/home/jair/Documents/arvato_project/process_and_ml/data_wrangler.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = df[key].replace(value, np.nan)


In [10]:
#clean_azdias, clean_customer, clean_test

In [11]:
from models.pipeline import preprocessing_baseline, show_metrics_baseline
from catboost import CatBoostClassifier, Pool
import pandas as pd

In [12]:
from catboost.utils import get_gpu_device_count
print('I see %i GPU devices' % get_gpu_device_count())


I see 1 GPU devices


In [13]:
df_to_model = pd.concat(
    [dfs_dict['customers_df'].assign(is_customer=1),
     dfs_dict['azdias'].assign(is_customer=0)]
)


In [14]:
cat_features = df_to_model.select_dtypes(include=['category', 'object']).columns

In [18]:
features, labels = preprocessing_baseline(df_to_model,
                                          cat_features=cat_features,
                                          target='is_customer')

X_train, X_test, X_valid = features
y_train, y_test, y_valid = labels

class_weights = (1, sum(y_train == 0) / sum(y_train == 1))



In [19]:
catboost = CatBoostClassifier(num_trees=40,
                              task_type='GPU',
                              max_depth=3,
                              min_child_samples=30,
                              one_hot_max_size=5,
                              class_weights=class_weights,
                              cat_features=cat_features,
                              random_state=RANDOM_STATE,
                              eval_metric='AUC',
                              od_type='Iter',
                              od_wait=40)


In [20]:
catboost.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)

CatBoostError: Invalid type for cat_feature[non-default value idx=2,feature_idx=10]=8.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [ ]:
show_metrics_baseline(catboost, features=features, labels=labels)


In [ ]:
#{"username":"jairai","key":"d602982bb910e207d33e9eebd8a9cf7e"}
#Predict the testfile

